In [1]:
import os
import numpy as np
from PIL import Image, ImageFile
from skimage import io
import matplotlib.pyplot as plt
import tifffile
from scipy import misc
from tifffile import TiffFile
from tifffile.tifffile import TIFF_DECOMPESSORS
from visualization import ids_to_greyscale, load_depth, show_bounding_boxes, load_stencil_ids, load_stencil_flags, \
get_bounding_boxes, show_loaded_bounding_boxes, get_detections, get_detections_multiple
from math import *
from shutil import copyfile
import progressbar
import pickle
import itertools

In [2]:
import visualization
visualization.multi_page = False
visualization.ini_file = 'gta-postprocessing.local.ini'

conn = visualization.get_connection()
cur = conn.cursor()
cur.execute("""SELECT snapshot_id, imagepath, cam_near_clip, cam_far_clip, timestamp, view_matrix, proj_matrix, world_matrix, 
    width, height, \
    ARRAY[st_x(camera_relative_rotation), st_y(camera_relative_rotation), st_z(camera_relative_rotation)] as relative_cam_rot, \
    ARRAY[st_x(camera_pos), st_y(camera_pos), st_z(camera_pos)] as camera_pos, \
    ARRAY[st_x(camera_rot), st_y(camera_rot), st_z(camera_rot)] as camera_rot, \
    ARRAY[st_x(camera_direction), st_y(camera_direction), st_z(camera_direction)] as camera_direction \
    FROM snapshots \
    WHERE run_id >= 47 \
    ORDER BY snapshot_id ASC \
    """)

results = [dict(res) for res in cur]
for i, res in enumerate(results):
    res['view_matrix'] = np.array(res['view_matrix'], dtype=np.float64)
    res['world_matrix'] = np.array(res['world_matrix'], dtype=np.float64)
    res['proj_matrix'] = np.array(res['proj_matrix'], dtype=np.float64)
    res['relative_cam_rot'] = np.array(res['relative_cam_rot'], dtype=np.float64)
    res['camera_pos'] = np.array(res['camera_pos'], dtype=np.float64)
    res['camera_rot'] = np.array(res['camera_rot'], dtype=np.float64)
    res['camera_direction'] = np.array(res['camera_direction'], dtype=np.float64)
    results[i] = res

print(len(results))


widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ',
            progressbar.FileTransferSpeed()]


pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(results)).start()
counter = 0

detections = get_detections_multiple(["'{}'".format(res['imagepath']) for res in results])
#for res in results:
#    res['detections'] = get_detections(res['imagepath'])
#    counter += 1
#    pbar.update(counter)
#pbar.finish()


key = lambda det: det['imagepath']
detections.sort(key=key)

checking = {key: len(list(group)) for key, group in itertools.groupby(detections, key=key)}
# print(checking)
print(checking.keys())

detection_groups = {key: list(group) for key, group in itertools.groupby(detections, key=key)}
for res in results:
    detections = detection_groups[res['imagepath']]
    res['detections'] = detections
    counter += 1
    pbar.update(counter)
pbar.finish()
    
with open('data_pickle.rick', mode='wb+') as file:
    pickle.dump(results, file)

820


dict_keys(['2018-02-25--20-03-37--068', '2018-02-25--20-06-54--893', '2018-02-25--19-53-45--872', '2018-02-25--20-12-43--532', '2018-02-22--20-09-19--185', '2018-02-25--20-06-25--148', '2018-02-25--20-06-30--507', '2018-02-25--19-58-15--549', '2018-02-25--19-59-57--736', '2018-02-25--20-10-50--206', '2018-02-25--20-00-00--453', '2018-02-25--20-00-19--928', '2018-02-25--20-06-03--325', '2018-02-25--20-11-26--476', '2018-02-25--20-03-45--114', '2018-02-25--20-04-39--096', '2018-02-25--20-12-12--226', '2018-02-25--19-53-29--880', '2018-02-25--20-05-35--813', '2018-02-25--20-09-37--834', '2018-02-25--20-00-45--651', '2018-02-25--20-01-10--819', '2018-02-25--20-06-19--841', '2018-02-25--19-59-29--458', '2018-02-25--19-58-12--315', '2018-02-25--20-12-29--781', '2018-02-25--19-53-34--966', '2018-02-25--20-05-53--916', '2018-02-25--20-10-56--080', '2018-02-25--20-09-00--114', '2018-02-26--12-36-49--675', '2018-02-25--20-07-36--794', '2018-02-25--20-02-09--192', '2018-02-25--20-07-15--717', '20

100% 820 |########################################################|  53.19  B/s


In [3]:
in_directory = 'D:\projekty\GTA-V-extractors\output'
out_directory = 'D:\projekty\GTA-V-extractors\output-to-datagrid'

widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ',
            progressbar.FileTransferSpeed()]


pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(results)).start()
counter = 0


for res in results:
    copyfile(os.path.join(in_directory, res['imagepath']+'.tiff'), os.path.join(out_directory, res['imagepath']+'.tiff'))
    copyfile(os.path.join(in_directory, res['imagepath']+'-depth.tiff'), os.path.join(out_directory, res['imagepath']+'-depth.tiff'))
    copyfile(os.path.join(in_directory, res['imagepath']+'-stencil.tiff'), os.path.join(out_directory, res['imagepath']+'-stenci.tiff'))
    counter += 1
    pbar.update(counter)
pbar.finish()


100% 820 |########################################################|  23.64  B/s


In [4]:
with open('data_pickle.rick', mode='rb') as file:
    results = pickle.load(file)
    print('I\'m pickle Rick!!!')
    for res in results:
        detections = list(detection_groups[res['imagepath']])
        print(len(detections))


I'm pickle Rick!!!
157
158
159
162
167
169
171
171
173
173
174
174
162
164
165
167
170
171
171
172
173
172
173
173
28
33
36
39
45
47
50
55
68
67
67
69
71
74
75
79
81
84
84
84
89
89
92
94
95
96
98
101
103
107
109
116
117
117
121
121
121
121
124
121
125
124
125
125
125
124
126
126
130
132
135
135
134
135
139
139
138
139
143
145
145
148
147
147
147
149
149
148
150
148
149
149
153
153
156
159
159
158
160
160
158
157
162
161
161
163
165
164
164
164
165
126
126
128
130
132
135
135
138
139
140
142
145
145
146
146
146
145
145
144
145
146
148
148
147
146
147
147
147
148
148
138
138
137
137
137
137
137
139
140
140
141
144
143
144
140
140
141
142
143
144
144
143
144
143
144
143
143
144
146
145
144
144
145
147
147
147
146
147
147
147
146
147
147
148
146
147
147
148
146
147
147
149
149
150
149
149
150
151
152
152
153
152
151
152
155
153
154
152
152
152
151
151
150
151
153
153
153
152
154
154
156
153
155
155
155
152
153
152
152
151
152
151
151
151
151
150
151
149
153
152
154
155
154
153
152
155
156


In [5]:
print(checking)
for res in results:
    detections = list(detection_groups[res['imagepath']])
    print(len(detections))


{'2018-02-25--20-03-37--068': 143, '2018-02-25--20-06-54--893': 145, '2018-02-25--19-53-45--872': 150, '2018-02-25--20-12-43--532': 184, '2018-02-22--20-09-19--185': 173, '2018-02-25--20-06-25--148': 151, '2018-02-25--20-06-30--507': 149, '2018-02-25--19-58-15--549': 148, '2018-02-25--19-59-57--736': 149, '2018-02-25--20-10-50--206': 168, '2018-02-25--20-00-00--453': 151, '2018-02-25--20-00-19--928': 152, '2018-02-25--20-06-03--325': 143, '2018-02-25--20-11-26--476': 172, '2018-02-25--20-03-45--114': 145, '2018-02-25--20-04-39--096': 141, '2018-02-25--20-12-12--226': 170, '2018-02-25--19-53-29--880': 139, '2018-02-25--20-05-35--813': 143, '2018-02-25--20-09-37--834': 161, '2018-02-25--20-00-45--651': 155, '2018-02-25--20-01-10--819': 153, '2018-02-25--20-06-19--841': 145, '2018-02-25--19-59-29--458': 146, '2018-02-25--19-58-12--315': 145, '2018-02-25--20-12-29--781': 182, '2018-02-25--19-53-34--966': 145, '2018-02-25--20-05-53--916': 145, '2018-02-25--20-10-56--080': 167, '2018-02-25--

157
158
159
162
167
169
171
171
173
173
174
174
162
164
165
167
170
171
171
172
173
172
173
173
28
33
36
39
45
47
50
55
68
67
67
69
71
74
75
79
81
84
84
84
89
89
92
94
95
96
98
101
103
107
109
116
117
117
121
121
121
121
124
121
125
124
125
125
125
124
126
126
130
132
135
135
134
135
139
139
138
139
143
145
145
148
147
147
147
149
149
148
150
148
149
149
153
153
156
159
159
158
160
160
158
157
162
161
161
163
165
164
164
164
165
126
126
128
130
132
135
135
138
139
140
142
145
145
146
146
146
145
145
144
145
146
148
148
147
146
147
147
147
148
148
138
138
137
137
137
137
137
139
140
140
141
144
143
144
140
140
141
142
143
144
144
143
144
143
144
143
143
144
146
145
144
144
145
147
147
147
146
147
147
147
146
147
147
148
146
147
147
148
146
147
147
149
149
150
149
149
150
151
152
152
153
152
151
152
155
153
154
152
152
152
151
151
150
151
153
153
153
152
154
154
156
153
155
155
155
152
153
152
152
151
152
151
151
151
151
150
151
149
153
152
154
155
154
153
152
155
156
156
158
157
158
158

In [6]:
results[0]['imagepath']
list(detection_groups['2018-02-22--20-08-31--234'])

[{'bbox': array([[ inf,  inf],
         [-inf, -inf]]),
  'bbox3d': array([-0.55108798,  0.5579586 , -0.25000003,  0.25000003, -1.29999995,
          0.81999993]),
  'class': 'Unknown',
  'handle': 60418,
  'imagepath': '2018-02-22--20-08-31--234',
  'pos': array([-764.0378418 , -160.69898987,   37.54673386]),
  'snapshot_id': 6185,
  'type': 'person'},
 {'bbox': array([[ inf,  inf],
         [-inf, -inf]]),
  'bbox3d': array([-0.60951751,  0.60998106, -0.25000003,  0.25000003, -1.29999995,
          0.85849118]),
  'class': 'Unknown',
  'handle': 60674,
  'imagepath': '2018-02-22--20-08-31--234',
  'pos': array([-665.21002197,  -46.67126846,   38.89189529]),
  'snapshot_id': 6185,
  'type': 'person'},
 {'bbox': array([[ inf,  inf],
         [-inf, -inf]]),
  'bbox3d': array([-0.55108798,  0.5579586 , -0.25000003,  0.25000003, -1.29999995,
          0.81999993]),
  'class': 'Unknown',
  'handle': 61186,
  'imagepath': '2018-02-22--20-08-31--234',
  'pos': array([-655.23132324, -145.980